In [ ]:
from __future__ import absolute_import, print_function
from flytekit.common.tasks.presto_task import SdkPrestoTask
from flytekit.sdk.tasks import inputs
from flytekit.sdk.types import Types

%env FLYTE_AUTH_ASSUMABLE_IAM_ROLE=arn:aws:iam::173840052742:role/modelbuilderapibatchworker-staging
%env FLYTE_PLATFORM_URL=flyte-staging.lyft.net
                    

COUNT_BY_HR_SCHEMA = Types.Schema(
    [
        ("total_count", Types.Integer),
        ("hr", Types.Integer),
    ],
)

get_data_task = SdkPrestoTask(
    task_inputs=inputs(ds=Types.String),
    statement="""
    SELECT
        COUNT(*) as total_count, hr
    FROM
        hive.events.event_ride_dropped_off
    WHERE
        ds = '{{ .Inputs.ds }}'
    GROUP BY 
        hr
    """,
    output_schema=COUNT_BY_HR_SCHEMA,
    discoverable=True,
    discovery_version="1",
)

task_exec = get_data_task.register_and_launch(project="flytekit", domain="development", inputs={"ds": '2020-06-10'})
print("Created {}".format(task_exec.id))
task_exec.wait_for_completion()
print("Done!")

env: FLYTE_AUTH_ASSUMABLE_IAM_ROLE=arn:aws:iam::173840052742:role/modelbuilderapibatchworker-staging
env: FLYTE_PLATFORM_URL=flyte-staging.lyft.net


In [2]:
task_exec.outputs

{'results': Schema([name: "total_count"
 , name: "hr"
 ]) @ s3://lyft-modelbuilder/v6/dfyelradv6-maingetdatatask-0/ (read-only)}

In [7]:
import pandas as pd
from flytekit.common import utils as _utils
from flytekit.sdk import test_utils as _test_utils

def print_schema(schema):
    with _test_utils.LocalTestFileSystem() as sandbox:
        # load schema data
        schema.download()
        column_names = [k for k in COUNT_BY_HR_SCHEMA.columns.keys()]
        df = pd.read_parquet(schema.local_path, columns=column_names)
    print(df)
    
print_schema(task_exec.outputs['results'])

    total_count  hr
0         28566   3
1         35750  12
2         57582  19
3         53997  22
4         18806   5
5         47072  16
6         28434  11
7         57291  21
8         35121   1
9         50795  17
10        23870   4
11        44494  14
12         9214   8
13        13961   6
14        48692  23
15        12013   9
16        10924   7
17        47528  15
18        31953   2
19        54788  18
20        40832  13
21        39763   0
22        57391  20
23        19496  10
